## PROMPT
Generate a pipeline for databricks that takes data from an API (https://api.spacexdata.com/v3/launches), performs a basic transformation task, and posts the result to another API (https://httpbin.org/post). Add prints for debugging the activities and display the execution time of the entire cycle

In [0]:
# Databricks Notebook - Python

# --- Configuration ---
api_source_url = "https://api.spacexdata.com/v3/launches"
api_target_url = "https://httpbin.org/post"

# --- Imports ---
import requests
import json
import time
from datetime import datetime

# --- Helper Functions ---

def fetch_data(url):
    """Fetches data from the given API URL."""
    print(f"Fetching data from: {url}")
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        print(f"Successfully fetched {len(data)} records.")
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from {url}: {e}")
        return None

def transform_data(data):
    """Performs a basic transformation on the fetched data."""
    if not data:
        print("No data to transform.")
        return None

    transformed_data = []
    print("Starting data transformation...")
    for item in data:
        transformed_item = {
            "flight_number": item.get("flight_number"),
            "mission_name": item.get("mission_name"),
            "launch_date_utc": item.get("launch_date_utc"),
            "rocket_name": item.get("rocket", {}).get("rocket_name")
        }
        transformed_data.append(transformed_item)
    print(f"Successfully transformed {len(transformed_data)} records.")
    return transformed_data

def post_data(url, data):
    """Posts the given data to the target API URL."""
    if not data:
        print("No data to post.")
        return None

    print(f"Posting data to: {url}")
    try:
        headers = {'Content-Type': 'application/json'}
        response = requests.post(url, json=data, headers=headers)
        response.raise_for_status()
        print(f"Successfully posted data. Status code: {response.status_code}")
        print(f"Response content: {response.text}")
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error posting data to {url}: {e}")
        return None

# --- Main Pipeline ---

if __name__ == "__main__":
    start_time = time.time()
    print("--- Starting SpaceX Data Pipeline ---")

    # 1. Fetch Data from SpaceX API
    raw_data = fetch_data(api_source_url)

    if raw_data:
        # 2. Transform Data
        transformed_data = transform_data(raw_data)

        if transformed_data:
            # 3. Post Transformed Data to httpbin.org
            print("\n--- Posting Transformed Data ---")
            post_response = post_data(api_target_url, transformed_data)
            if post_response:
                print("\nData successfully posted to the target API.")
            else:
                print("\nFailed to post data to the target API.")
        else:
            print("\nNo data to post after transformation.")
    else:
        print("\nFailed to fetch data from the source API.")

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"\n--- Pipeline Finished ---")
    print(f"Execution Time: {execution_time:.2f} seconds")